In [1]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()


In [4]:


import xml.etree.ElementTree as ET

def process_text_data(text_data, File_name_to_be_moved, request_id, current_timestamp, post_cur, table_stg):
    print(f"Processing text data inside process_text_data")
    try:
        # Parse the XML data
        root = ET.fromstring(text_data)

        tag_to_column = {
            'BICEPSVENDOR': 'VENDOR_NUMBER',
            'SSAPERCENT': 'SSA_RATE',
            'SSABASIS': 'SSA_RATE_TYPE',
            'SSAACCOUNT': 'SSA_RATE_BASIS_ACC',
            'SSATYPE': 'SSA_RATE_BASIS_GROSS_FLAG',
            'TRANSACTIONID': 'TRANSACTIONID'
        }

        rows_count = 0

        # Loop through each <row> element in the XML
        for row in root.findall('row'):
            # Define a dictionary to store values
            element_values = {}

            # Extract values from the XML
            for tag, column_name in tag_to_column.items():
                element = row.find(tag)
                if element is not None and element.text is not None:
                    element_values[tag] = element.text.strip()

            # Handle 'BICEPSVENDOR' processing logic
            if 'BICEPSVENDOR' in element_values:
                bicepsvendor_value = element_values['BICEPSVENDOR']

                # Check if the value has at least 6 characters
                if len(bicepsvendor_value) >= 6:
                    # Extract the last 6 digits as the vendor number
                    vendor_number = bicepsvendor_value[-6:]

                    # Extract the remaining part to the left of the last 6 digits as the facility number
                    facility_value = bicepsvendor_value[:-6]

                    # Assign the extracted values to new keys
                    element_values['VENDOR_FACILITY'] = facility_value

                    # Update 'BICEPSVENDOR' to only contain the vendor number
                    element_values['BICEPSVENDOR'] = vendor_number
                else:
                    print('Error while extracting', element_values['BICEPSVENDOR'])

            # Replace column names with the ones from tag_to_column
            mapped_values = {tag_to_column[key]: value for key, value in element_values.items() if key in tag_to_column}

            # Include additional columns in the mapping
            mapped_values['SSA_RATE_BASIS_GROSS_FLAG'] = element_values.get('SSATYPE', '')
            mapped_values['REQUEST_ID'] = request_id
            mapped_values['CREATION_DATE'] = current_timestamp
            mapped_values['LAST_UPDATE_DATE'] = current_timestamp
            mapped_values['CREATED_BY'] = 'CSPOMS'
            mapped_values['LAST_UPDATED_BY'] = 'CSPOMS'
            mapped_values['FILE_NAME'] = File_name_to_be_moved
            mapped_values['PROCESS_STATUS'] = 'N'

            # Generate SQL INSERT statement
            columns_str = ",".join(mapped_values.keys())
            values_str = ",".join(f"'{v}'" for v in mapped_values.values())

            insert_query = f"INSERT INTO {table_stg} ({columns_str}) VALUES ({values_str});"
            post_cur.execute(insert_query)
            rows_count += post_cur.rowcount  # Increment the row count

        print(f"Processed file: {File_name_to_be_moved}, Number of rows inserted: {rows_count}")

    except Exception as e:
        print(f"Error processing text data:")
        print(f"Error: {e}")


# Example usage:
text_data = """<?xml version='1.0' encoding='utf-8'?>
<data>
  <row>
    <BICEPSVENDOR>904100629</BICEPSVENDOR>
    <SSAPERCENT>0.57</SSAPERCENT>
    <SSABASIS>G</SSABASIS>
    <SSAACCOUNT>S3</SSAACCOUNT>
    <SSATYPE>P</SSATYPE>
    <VENDORCHG/>
    <TRANSACTIONID>V0726240907060000000047</TRANSACTIONID>
  </row>
  <row>
    <BICEPSVENDOR>904100842</BICEPSVENDOR>
    <SSAPERCENT>1.68</SSAPERCENT>
    <SSABASIS>G</SSABASIS>
    <SSAACCOUNT>S3</SSAACCOUNT>
    <SSATYPE>P</SSATYPE>
    <VENDORCHG/>
    <TRANSACTIONID>V0726240907060000000048</TRANSACTIONID>
  </row>
  </data>
"""

# Call the function with sample data
process_text_data( text_data, "FileName.txt", "-999", "2024-09-04", post_cur, "xxpo_corp_incentive_ssa_stg")
connection.commit()

Processing text data inside process_text_data
Processed file: FileName.txt, Number of rows inserted: 2


In [90]:
# Example usage
request_id = -999
affected_rows = check_vendor_master(post_cur)
print(f"Number of rows updated: {affected_rows}")


Number of rows updated: 0


In [75]:


post_cur.execute(f"SELECT * FROM xxpo_corp_incentive_ssa_stg WHERE PROCESS_STATUS in ('N')")
upsert_rows = post_cur.fetchall()
upsert_rows
column_names = [desc[0] for desc in post_cur.description]
upsert_df = pd.DataFrame(upsert_rows, columns=column_names)
upsert_df = upsert_df.replace({np.nan: None})
# Convert vendor_facility and vendor_number to numeric, then back to string to remove leading zeros
upsert_df['vendor_facility'] = upsert_df['vendor_facility'].astype(int).astype(str)
upsert_df['vendor_number'] = upsert_df['vendor_number'].astype(int).astype(str)
# Convert 'bic_vendor_facility' to string and apply LPAD to 3 places
upsert_df['bic_vendor_facility'] = upsert_df['bic_vendor_facility'].astype(str).str.zfill(3)
# Ensure 'bic_vendor_number' is a string and apply LPAD to 6 places
upsert_df['bic_vendor_number'] = upsert_df['bic_vendor_number'].astype(str).str.zfill(6)

updated_df = upsert_df.drop_duplicates(subset=['vendor_facility', 'vendor_number'], keep='last')
updated_df
# upsert_vendor_master_and_costing_info(upsert_df,post_cur,request_id)

,seq_id,vendor_facility,vendor_number,ssa_rate,ssa_rate_type,ssa_rate_basis_acc,ssa_rate_basis_gross_flag,ssa_rate_basis_net_flag,request_id,file_name,creation_date,last_update_date,created_by,last_updated_by,process_status,error_message,transactionid
1,32,904,100842,1.68,G,S3,P,P,-999,FileName.txt,2024-09-04,2024-09-04,CSPOMS,CSPOMS,N,None,072624090706000000049
2,33,24,46076,4.35,G,S3,P,P,-999,FileName.txt,2024-09-04,2024-09-04,CSPOMS,CSPOMS,N,None,072624090706000000050
11,42,904,3628,2.0,R,S3,P,P,-999,FileName.txt,2024-09-04,2024-09-04,CSPOMS,CSPOMS,N,None,072624090706000000059


In [86]:
# upsert_vendor_master_and_costing_info(updated_df,post_cur,request_id)
update_vendor_master_and_costing_info(updated_df,post_cur,request_id)

100842 904 -999
1.68 G S3 P P -999
46076 24 -999
4.35 G S3 P P -999
3628 904 -999
2.0 R S3 P P -999


In [89]:
def check_vendor_master(post_cur):
    update_query = '''
    UPDATE xxpo_corp_incentive_ssa_stg stg
    SET 
        process_status = 'E',
        error_message = 'Record not exists in xxpo_vendor_master'
    WHERE
         process_status = 'N'
        AND
         NOT EXISTS (
            SELECT 1
            FROM xxpo_vendor_master xvm
            WHERE 
                 xvm.vendor_facility::numeric = stg.vendor_facility::numeric
                AND xvm.vendor_number::numeric = stg.vendor_number::numeric
        );
    '''
    
    # Execute the update statement
    post_cur.execute(update_query)
    
    # Get the count of affected rows
    affected_rows = post_cur.rowcount
    
    return affected_rows



In [44]:
def upsert_vendor_master_and_costing_info(merged_df, cur, request_id):
    merged_df = merged_df.replace({np.nan: None})

    for index, row in merged_df.iterrows():
        try:
            print(row['vendor_number'], row['vendor_facility'], request_id)
            cur.execute(
                '''
                INSERT INTO cspoms.XXPO_VENDOR_MASTER (
                    CS_VENDOR_ID,
                    VENDOR_NUMBER,
                    VENDOR_FACILITY,
                    REQUEST_ID,
                    CREATION_DATE,
                    LAST_UPDATE_DATE,
                    CREATED_BY,
                    LAST_UPDATED_BY
                ) 
                VALUES (
                    nextval('cs_vendor_id_seq'),
                    %s,
                    LPAD(%s::VARCHAR, 3, '0'),
                    %s,
                    current_timestamp,
                    current_timestamp,
                    'CSPOMS',
                    'CSPOMS'
                ) 
                ON CONFLICT (VENDOR_NUMBER, VENDOR_FACILITY) DO UPDATE SET 
                    VENDOR_NUMBER = EXCLUDED.VENDOR_NUMBER,
                    VENDOR_FACILITY = LPAD(EXCLUDED.VENDOR_FACILITY::VARCHAR, 3, '0'),
                    REQUEST_ID = EXCLUDED.REQUEST_ID,
                    LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                    LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                RETURNING CS_VENDOR_ID
                ''',
                (
                    row['vendor_number'],
                    row['vendor_facility'],
                    request_id
                )
            )

            # Fetch the generated CS_VENDOR_ID
            cs_vendor_id = cur.fetchone()[0]

            if cs_vendor_id:
                print(row['ssa_rate'], row['ssa_rate_type'], row['ssa_rate_basis_acc'], row['ssa_rate_basis_gross_flag'], row['ssa_rate_basis_net_flag'], request_id)
                cur.execute(
                    '''
                    INSERT INTO XXPO_VENDOR_COSTING_INFO (
                        CS_VENDOR_ID,
                        SSA_RATE,
                        SSA_RATE_TYPE,
                        SSA_RATE_BASIS_ACC,
                        SSA_RATE_BASIS_GROSS_FLAG,
                        SSA_RATE_BASIS_NET_FLAG,
                        REQUEST_ID,
                        CREATION_DATE, 
                        LAST_UPDATE_DATE, 
                        CREATED_BY, 
                        LAST_UPDATED_BY
                    )
                    VALUES (
                        %s,
                        %s,
                        %s,
                        %s,
                        %s,
                        %s,
                        %s,
                        current_timestamp,
                        current_timestamp,
                        'CSPOMS',
                        'CSPOMS'
                    )
                    ON CONFLICT (CS_VENDOR_ID) DO UPDATE SET
                        SSA_RATE = EXCLUDED.SSA_RATE,
                        SSA_RATE_TYPE = EXCLUDED.SSA_RATE_TYPE,
                        SSA_RATE_BASIS_ACC = EXCLUDED.SSA_RATE_BASIS_ACC,
                        SSA_RATE_BASIS_GROSS_FLAG = EXCLUDED.SSA_RATE_BASIS_GROSS_FLAG,
                        SSA_RATE_BASIS_NET_FLAG = EXCLUDED.SSA_RATE_BASIS_NET_FLAG,
                        REQUEST_ID = EXCLUDED.REQUEST_ID,
                        LAST_UPDATE_DATE = EXCLUDED.LAST_UPDATE_DATE,
                        LAST_UPDATED_BY = EXCLUDED.LAST_UPDATED_BY
                    ''',
                    (
                        cs_vendor_id,
                        row['ssa_rate'],
                        row['ssa_rate_type'],
                        row['ssa_rate_basis_acc'],
                        row['ssa_rate_basis_gross_flag'],
                        row['ssa_rate_basis_net_flag'],
                        request_id
                    )
                )

        except Exception as e:
            print(f"An error occurred while inserting to Base Tables: {e}, {row}")
            raise Exception('Error occurred while inserting to Base Tables', e)


In [84]:
def update_vendor_master_and_costing_info(merged_df, cur, request_id):
    # Replace NaN values with None to handle them correctly in the database
    merged_df = merged_df.replace({np.nan: None})

    for index, row in merged_df.iterrows():
        try:
            print(row['vendor_number'], row['vendor_facility'], request_id)

            # Update statement for XXPO_VENDOR_MASTER table
            cur.execute(
                '''
                UPDATE cspoms.XXPO_VENDOR_MASTER
                SET 
                    REQUEST_ID = %s,
                    LAST_UPDATE_DATE = current_timestamp,
                    LAST_UPDATED_BY = 'CSPOMS'
                WHERE 
                    VENDOR_NUMBER = %s
                    AND VENDOR_FACILITY = LPAD(%s::VARCHAR, 3, '0')
                RETURNING CS_VENDOR_ID
                ''',
                (
                    request_id,
                    row['vendor_number'],
                    row['vendor_facility']
                )
            )

            # Fetch the updated CS_VENDOR_ID if the update was successful
            cs_vendor_id = cur.fetchone()[0]

            if cs_vendor_id:
                print(row['ssa_rate'], row['ssa_rate_type'], row['ssa_rate_basis_acc'], row['ssa_rate_basis_gross_flag'], row['ssa_rate_basis_net_flag'], request_id)

                # Update statement for XXPO_VENDOR_COSTING_INFO table
                cur.execute(
                    '''
                    UPDATE XXPO_VENDOR_COSTING_INFO
                    SET 
                        SSA_RATE = %s,
                        SSA_RATE_TYPE = %s,
                        SSA_RATE_BASIS_ACC = %s,
                        SSA_RATE_BASIS_GROSS_FLAG = %s,
                        SSA_RATE_BASIS_NET_FLAG = %s,
                        REQUEST_ID = %s,
                        LAST_UPDATE_DATE = current_timestamp,
                        LAST_UPDATED_BY = 'CSPOMS'
                    WHERE 
                        CS_VENDOR_ID = %s
                    ''',
                    (
                        row['ssa_rate'],
                        row['ssa_rate_type'],
                        row['ssa_rate_basis_acc'],
                        row['ssa_rate_basis_gross_flag'],
                        row['ssa_rate_basis_net_flag'],
                        request_id,
                        cs_vendor_id
                    )
                )

        except Exception as e:
            print(f"An error occurred while updating the Base Tables: {e}, {row}")
            raise Exception('Error occurred while updating the Base Tables', e)


In [87]:
connection.commit()